In [ ]:
# Nueva importación modularizada y simplificada
from src.preprocessing import load_data, clean_data, feature_engineering, scale_and_split

In [ ]:
data_path = "../data/raw/sensor_data.csv"
df = load_data(data_path)  # Usa función modularizada

,timestamp,temperatura,vibracion,presion,falla
0,2024-01-01 00:00:00,72.48,2.79,28.64,0
1,2024-01-01 01:00:00,69.31,2.77,29.39,0
2,2024-01-01 02:00:00,73.24,2.10,28.81,0
3,2024-01-01 03:00:00,77.62,2.83,30.22,0
4,2024-01-01 04:00:00,68.83,3.37,32.39,0


In [ ]:
df = clean_data(df)  # Función modularizada realiza esto automáticamente

✅ Confirmado: No existen datos faltantes en el dataset.


In [ ]:
df = feature_engineering(df)  # Ingeniería de características modularizada

,temperatura,vibracion,presion,falla,hora,dia_semana
0,72.48,2.79,28.64,0,0,0
1,69.31,2.77,29.39,0,1,0
2,73.24,2.10,28.81,0,2,0
3,77.62,2.83,30.22,0,3,0
4,68.83,3.37,32.39,0,4,0


In [ ]:
# División y escalado modularizados en una sola función
X_train_scaled, X_test_scaled, y_train, y_test = scale_and_split(df, target='falla')

Tamaño de características (X): (5000, 5)
Tamaño de variable objetivo (y): (5000,)


In [9]:
# Rutas para guardar los datos procesados
processed_path = "../data/processed/"
os.makedirs(processed_path, exist_ok=True)  # Crea carpeta si no existe

# Guardamos los datos procesados
X_train_scaled.to_csv(processed_path + 'X_train_scaled.csv', index=False)
X_test_scaled.to_csv(processed_path + 'X_test_scaled.csv', index=False)
y_train.to_csv(processed_path + 'y_train.csv', index=False)
y_test.to_csv(processed_path + 'y_test.csv', index=False)

print("✅ Datos procesados guardados correctamente.")

✅ Datos procesados guardados correctamente.
